# Working with functions in Azure OpenAI
This notebook shows how to use the Chat Completions API in combination with functions to extend the current capabilities of GPT models. GPT models, do not inherently support real-time interaction with external systems, databases, or files. However, functions can be used to do so.

Overview: <br>
`tools` (previously called `functions`) is an optional parameter in the Chat Completion API which can be used to provide function specifications. This allows models to generate function arguments for the specifications provided by the user. 

Note: The API will not execute any function calls. Executing function calls using the outputed argments must be done by developers. 

## Local function definition

In [11]:
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

### Define a helper function to validate the function call
It's possible that the models could generate incorrect function calls so it's important to validate the calls. Here we define a simple helper function to validate the function call although you could apply more complex validation for your use case.

In [12]:
import inspect
# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

## Get the function(s) name(s)
This Python code is using the OpenAI API to generate a response to a user's question about the weather in San Francisco and Paris. It sends a conversation message and a tool (a function to get the current weather) to the API. The API then generates a response.

In [ ]:
# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": "What's the weather like San Francisco and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

response_json = json.loads(response_message.model_dump_json())
print(json.dumps(response_json, indent=4))

# Getting final response 
Process the response by invoking the appropriate function(s) with the arguments obtained from the initial response

In [ ]:

# Step 2: check if the model wanted to call a function
if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_json)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        if check_args(function_to_call, function_args) is False:
            print("Invalid number of arguments for function: " + function_name)
            break
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    second_response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        messages=messages,
    )  # get a new response from the model where it can see the function response
    
    second_response_json = json.loads(second_response.model_dump_json())
    print(json.dumps(second_response_json, indent=4))

In [ ]:
print (messages)

In [ ]:
print (messages)